## Term Weighting and VSM (Assignment 2)

Name : Suyash Chintawar

Roll No. : 191IT109



In [1]:
# Install required external libraries
!pip install requests
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import re
import os
import nltk
import pandas as pd
import numpy as np
import requests
from scipy import spatial
from bs4 import BeautifulSoup
from collections import defaultdict

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [3]:
# Get links from where the documents will be extracted

root = 'https://www.incredibleindia.org/'
cities = ['delhi','bengaluru','kolkata','varanasi','hyderabad','jaipur','udaipur']
links = []

for city in cities:
  city_page_url = 'https://www.incredibleindia.org/content/incredible-india-v2/en/destinations/' + city + '.html'
  page = requests.get(city_page_url)
  soup = BeautifulSoup(page.content, 'html.parser')

  s = soup.find_all('div', class_='immersive-experience-item')
  for category in s:
    link = category.find('a')['href']
    link = root + link
    links.append(link)

In [4]:
links

['https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/delhi/heritage.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/delhi/spiritual.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/delhi/food-and-cuisine.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/bengaluru/nature.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/bengaluru/food-and-cuisine.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/bengaluru/art-and-crafts.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/kolkata/food-and-cuisine.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/kolkata/nature.html',
 'https://www.incredibleindia.org//content/incredible-india-v2/en/destinations/kolkata/arts-and-crafts.html',
 'https://www.incredibleindia.org//cont

In [5]:
!rm -rf data
!mkdir data

In [6]:
# Extracting the documents from the links above

count = 0
for url in links:
  page = requests.get(url)
  soup = BeautifulSoup(page.content,'html.parser')
  s = soup.find_all('div',class_='listicle aem-GridColumn aem-GridColumn--default--12')
  for item in s:
    title = item.find('h2',class_='title').text
    title = re.sub('[^a-zA-Z0-9 ]','',title)
    title = '-'.join(title.lower().split())
    content = item.find('p').text
    content = ' '.join(content.split())

    ofile = open('data/'+title+'.txt','w')
    ofile.write(content)
    ofile.close()

    count+=1

print('Number of files generated:',count)

Number of files generated: 151


In [1]:
# Fetch documents if already extracted and stored
!gdown https://drive.google.com/uc?id=1mm4N8w7Is2TKtuCnX0j8idPi3KGbY3X8
!unzip 'data.zip'

Downloading...
From: https://drive.google.com/uc?id=1mm4N8w7Is2TKtuCnX0j8idPi3KGbY3X8
To: /content/data.zip
100% 79.3k/79.3k [00:00<00:00, 89.2MB/s]
Archive:  data.zip
   creating: data/
  inflating: data/bagore-ki-haveli.txt  
  inflating: data/durga-mandir.txt   
  inflating: data/safed-maas.txt     
  inflating: data/kathi-rolls.txt    
  inflating: data/mekedatu.txt       
  inflating: data/jewellery.txt      
  inflating: data/parliament-house.txt  
  inflating: data/jagdish-temple.txt  
  inflating: data/cubbon-park.txt    
  inflating: data/the-lake-palace.txt  
  inflating: data/teen-murti-house.txt  
  inflating: data/mirchi-ka-salaan.txt  
  inflating: data/hesaraghatta-lake.txt  
  inflating: data/parantha.txt       
  inflating: data/fish-finger.txt    
  inflating: data/alipore-zoological-garden.txt  
  inflating: data/osmania-biscuit.txt  
  inflating: data/mukutmanipur.txt   
  inflating: data/hyderabadi-haleem.txt  
  inflating: data/jag-mandir-palace.txt  
  inflating:

In [10]:
# Part A
# Preprocessing using different techniques

#Get file names and their texts
path = 'data/'
files = []
texts = []
for file_name in os.listdir(path):
  file = open(path+file_name,'r')
  files.append(file_name)
  texts.append(file.read())

files = files[:100]
texts = texts[:100]

print('Files: ',files)
print('Files length:',len(files))
print('Texts: ',texts)
print('Texts length:',len(texts))

Files:  ['bagore-ki-haveli.txt', 'durga-mandir.txt', 'safed-maas.txt', 'kathi-rolls.txt', 'mekedatu.txt', 'jewellery.txt', 'parliament-house.txt', 'jagdish-temple.txt', 'cubbon-park.txt', 'the-lake-palace.txt', 'teen-murti-house.txt', 'mirchi-ka-salaan.txt', 'hesaraghatta-lake.txt', 'parantha.txt', 'fish-finger.txt', 'alipore-zoological-garden.txt', 'osmania-biscuit.txt', 'mukutmanipur.txt', 'hyderabadi-haleem.txt', 'jag-mandir-palace.txt', 'sandesh.txt', 'whole-goat-roasted.txt', 'connaught-place.txt', 'ghugni.txt', 'mandawa-fort.txt', 'blue-pottery.txt', 'jute-decor.txt', 'manikarnika-ghat.txt', 'mecca-masjid.txt', 'mysore-pak.txt', 'kadhi.txt', 'phuchka.txt', 'conch-shell-craft.txt', 'jaigarh-fort.txt', 'mutton-biryani.txt', 'chhole-bhature.txt', 'muthyala-maduvu-or-pearl-valley.txt', 'haldighati.txt', 'safdarjung-fort.txt', 'assi-ghat.txt', 'baati-chokha.txt', 'butter-chicken.txt', 'national-war-memorial.txt', 'gujiya.txt', 'vada.txt', 'rasogolla.txt', 'malkha-work.txt', 'iskcon-te

In [11]:
# Counting total index terms in the corpus
def get_all_tokens(texts,count=True):
  all_tokens = set()
  for text in texts:
    for token in text:
      all_tokens.add(token)
  if count:
    return len(all_tokens)
  else:
    return all_tokens

In [12]:
# Tokenization
def tokenize(text):
  return text.split()

In [13]:
# Nromalization
def normalize(text):
  new_tokens = []
  for token in text:
    token = re.sub('[^a-zA-Z0-9]','',token)
    token = token.lower()
    new_tokens.append(token)
  return new_tokens

In [14]:
from nltk.stem import WordNetLemmatizer

# Lemmatization
def lemmatize(text):
  lemmatized_text = []
  lemmatizer = WordNetLemmatizer()
  for token in text:
    lemmatized_token = lemmatizer.lemmatize(token)
    lemmatized_text.append(lemmatized_token)
  return lemmatized_text

In [15]:
from nltk.corpus import stopwords

# Stopword removal
def remove_stopwords(text):
  stopwords_list = stopwords.words('english')
  new_tokens = []
  for token in text:
    if token not in stopwords_list:
      new_tokens.append(token)
  return new_tokens

In [16]:
tokenized_texts = []
for text in texts:
  tokenized_texts.append(tokenize(text))
print('Tokenized texts:\n',tokenized_texts)
print('Number of tokens after tokenization:',get_all_tokens(tokenized_texts),end='\n\n')

normalized_texts = []
for text in tokenized_texts:
  normalized_texts.append(normalize(text))
print('Normalized texts:\n',normalized_texts)
print('Number of tokens after normalization:',get_all_tokens(normalized_texts),end='\n\n')

lemmatized_texts = []
for text in normalized_texts:
  lemmatized_texts.append(lemmatize(text))
print('Lemmatized texts:\n',lemmatized_texts)
print('Number of tokens after lemmatization:',get_all_tokens(lemmatized_texts),end='\n\n')

preprocessed_texts = []
for text in lemmatized_texts:
  preprocessed_texts.append(remove_stopwords(text))
print('Texts after stopword removal:\n',preprocessed_texts)
print('Number of tokens after lemmatization:',get_all_tokens(preprocessed_texts),end='\n\n')


Tokenized texts:
 [['Standing', 'on', 'the', 'waterfront', 'of', 'Lake', 'Pichola', 'at', 'Gangori', 'Ghat', 'is', 'the', 'stunning', 'Bagore', 'ki', 'Haveli.', 'Constructed', 'in', 'the', '18th', 'century', 'by', 'Amar', 'Chand', 'Badwa,', 'the', 'then', 'Prime', 'Minister', 'of', 'Mewar', 'kingdom,', 'Bagore', 'Ki', 'Haveli', 'was', 'a', 'private', 'property', 'till', "India's", 'independence.', 'Today,', 'this', 'mansion', 'with', 'ostentatious', 'architecture,', 'is', 'a', 'museum.', 'Reflecting', 'Mewar’s', 'rich', 'heritage,', 'Bagore', 'ki', 'Haveli', 'has', 'an', 'assortment', 'of', 'huge', 'courtyards,', 'balconies,', 'jharokhas,', 'archways,', 'cupolas', 'and', 'a', 'fountain.', 'With', 'around', '138', 'rooms,', 'the', 'interiors', 'of', 'the', 'Haveli', 'are', 'decorated', 'with', 'alluring', 'glasswork', 'and', 'murals,', 'including', 'the', 'chambers', 'of', 'the', 'royal', 'ladies,', 'renowned', 'for', 'intricate', 'stained', 'glass', 'windows.'], ['known', 'as', 'Monkey

In [17]:
# List of all documents each represented in a list of index terms
documents = preprocessed_texts

# Set of all index terms across all documents
tokens = get_all_tokens(preprocessed_texts,False)
print('Total no. of index terms:',len(tokens))

Total no. of index terms: 2652


### Part A - Generating term weights for the vocabulary

In [18]:
# Computing term frequency using two methods, log normzalization and double normalization-K
def tf(term, document,tf_type='logNormal',k=0.3):
  '''
  This function computes term freq for a index term in a document
  '''
  freq = document.count(term)
  if tf_type=='logNormal' and freq > 0:
    tf = np.log2(freq) + 1
    return tf
  # elif freq > 0 and tf_type=='doubleNormalK':
  elif tf_type=='doubleNormalK':
    max_count = document.count(max(set(document), key=document.count))
    tf = k + (1-k)*(freq/max_count)
    return tf
  else:
    return 0

In [19]:

def idf(term,documents,idf_type='invFreq'):
  '''
  This function computes inverse doc freq for a index term for all documents
  Three methods used,
  1) inverse frequency -> default
  2) inverse frequency smooth
  3) probabilistic inverse frequency
  '''
  N = len(documents)
  ni = 0
  for document in documents:
    if term in document:
      ni += 1
  if idf_type=='invFreq':
    return np.log2(N/ni) 
  elif idf_type=='invFreqSmooth':
    return np.log2(1 + (N/ni))
  elif idf_type=='invFreqProb':
    return np.log2((N-ni)/ni)

In [20]:
def get_tf(documents,tokens,tf_type='logNormal'):
  '''
  This function computes term freq for each (index term, document) pair
  Returns a matrix of dimension (m,n), where
  'm'-> no. of docs
  'n'-> no. of index terms
  '''
  term_freq = []
  for document in documents:
    tf_doc = []
    for token in tokens:
      tf_doc.append(tf(token,document,tf_type))
    term_freq.append(tf_doc)

  return term_freq

In [21]:
def get_idf(documents,tokens,idf_type='invFreq'):
  '''
  This function computes index doc freq for each index term
  Returns a matrix of dimension (1,n), where
  'n'-> no. of index terms
  '''
  inverse_doc_freq = []

  for token in tokens:
    inverse_doc_freq.append(idf(token,documents,idf_type))

  return inverse_doc_freq

In [22]:
def get_term_weights(documents, tokens, tf_type='logNormal', idf_type='invFreq'):
  '''
  Computes term weights (TF-IDF) for each (index term, document) pair
  Returns a matrix of dimension (m,n), where
  'm'-> no. of docs
  'n'-> no. of index terms
  '''
  term_weights = get_tf(documents,tokens,tf_type)
  term_weights = np.array(term_weights,dtype=float)
  idf = get_idf(documents,tokens,idf_type)
  idf=np.array(idf,dtype=float)
  for item in idf:
    if item<0:
      print(item)

  for i in range(len(term_weights)):
    term_weights[i] = np.multiply(term_weights[i],idf)
  
  return term_weights

In [23]:
# Euclidean distance between two numpy arrays
def euclidean_dist(a,b):
  return  np.linalg.norm(a-b)

# Cosine similarity between two numpy arrays
def cosine_sim(a,b):
  return  1 - spatial.distance.cosine(a,b)


In [24]:
def generate_ranked_list(documents,tokens,term_weights,query,measure='euclidean',tf_type='logNormal',idf_type='invFreq'):
  '''
  Generates rankings for documents w.r.t the passed query
  Returns a list of (distance metric value, document ID) pair sorted according to highest to lowest ranks
  '''
  ranked_list = []

  # represent query in same vector space
  tf_query = []
  for token in tokens:
    tf_query.append(tf(token,query,tf_type))
  
  term_weight_query = np.array(tf_query,dtype=float)
  idf = get_idf(documents,tokens,idf_type)
  idf=np.array(idf,dtype=float)
  term_weight_query = np.multiply(term_weight_query,idf)  

  # compare with distance measure
  if measure=='euclidean':
    for i in range(len(term_weights)):
      distance = euclidean_dist(term_weight_query,term_weights[i])
      ranked_list.append([distance,i])
    
    ranked_list.sort()
    return ranked_list

  elif measure=='cosine':
    for i in range(len(term_weights)):
      similarity = cosine_sim(term_weight_query,term_weights[i])
      ranked_list.append([similarity,i])
    
    ranked_list.sort(reverse=True)
    return ranked_list

### Part B - Generate ranked list for sample queries using a distance and a similarity measure

1) Using Euclidean distance as distance measure using standard TF-IDF weighting scheme

In [37]:
# document representation
term_weights = get_term_weights(documents,tokens,'logNormal','invFreq')
print('Dimensions of generated matrix:',term_weights.shape)
term_weights = np.array(term_weights)
df=pd.DataFrame(data=term_weights[0:,0:], index=[i for i in range(term_weights.shape[0])], columns=tokens)
df

Dimensions of generated matrix: (100, 2652)


,,sambhar,ground,staple,construction,heart,businessmen,include,governance,afghan,...,sword,birdlovers,surrounding,comprising,tall,thereby,doused,upper,dialect,court
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.643856,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,5.643856,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
#Assuming query terms occur in vocab
queries = [['deepfried','flatbread'],['standing','waterfront','pichola'], ['traditional','curry','pakoda','boiled','rice']]

for idx, query in enumerate(queries):
  rankings = generate_ranked_list(preprocessed_texts,tokens,term_weights,query,'euclidean') 
  print('********************************')
  print('QUERY NO:',idx,'\nQUERY:', query, '\n\nTOP 10 RANKINGS:',end='\n\n')
  print('Document ID\tEuclidean Distance from query')
  for i in range(10):
    print(rankings[i][1],'\t\t',rankings[i][0])
  
  print('\nTOP 3 RANKED DOCUMENTS:')
  for i in range(3):
    print('Document ID:', rankings[i][1])
    print('Content:\n',' '.join(documents[rankings[i][1]]),end='\n\n')
  print('********************************\n')

********************************
QUERY NO: 0 
QUERY: ['deepfried', 'flatbread'] 

TOP 10 RANKINGS:

Document ID	Euclidean Distance from query
92 		 16.598756322494378
94 		 18.693466571080926
30 		 19.050018404672496
21 		 19.955239333959053
96 		 20.21986732106013
64 		 20.78139648459215
44 		 21.285727113878064
23 		 21.898325659033606
91 		 22.811620305237366
14 		 24.85070060976365

TOP 3 RANKED DOCUMENTS:
Document ID: 92
Content:
 deepfried flatbread made wheat generous amount ghee clarified butter eaten curry especially fish curry

Document ID: 94
Content:
 located ajmer road small village located35 km jaipur known traditional bagru print

Document ID: 30
Content:
 kadhi traditional curdbased curry fried gram flour fritter called pakoda often eaten boiled rice roti indian flatbread

********************************

********************************
QUERY NO: 1 
QUERY: ['standing', 'waterfront', 'pichola'] 

TOP 10 RANKINGS:

Document ID	Euclidean Distance from query
94 		 20.1307

2) Using Cosine similarity as similarity measure using standard TF-IDF weighting scheme

In [27]:
# document representation
term_weights = get_term_weights(documents,tokens,'logNormal','invFreq')
print('Dimensions of generated matrix:',term_weights.shape)

Dimensions of generated matrix: (100, 2652)


In [28]:
#Assuming query terms occur in vocab
queries = [['deepfried','flatbread'],['standing','waterfront','pichola'], ['traditional','curry','pakoda','boiled','rice']]

for idx, query in enumerate(queries):
  rankings = generate_ranked_list(preprocessed_texts,tokens,term_weights,query,'cosine') 
  print('********************************')
  print('QUERY NO:',idx,'\nQUERY:', query, '\n\nTOP 10 RANKINGS:',end='\n\n')
  print('Document ID\tCosine Similarity with query')
  for i in range(10):
    print(rankings[i][1],'\t\t',rankings[i][0])
  
  print('\nTOP 3 RANKED DOCUMENTS:')
  for i in range(3):
    print('Document ID:', rankings[i][1])
    print('Content:\n',' '.join(documents[rankings[i][1]]),end='\n\n')
  print('********************************\n')

********************************
QUERY NO: 0 
QUERY: ['deepfried', 'flatbread'] 

TOP 10 RANKINGS:

Document ID	Cosine Similarity with query
23 		 0.5106944347780581
92 		 0.3516205550719911
44 		 0.19027931230030726
30 		 0.11374068581147201
91 		 0.09448419581652645
3 		 0.07792131089792209
85 		 0.04718540515796432
41 		 0.04530916930921902
13 		 0.03842470377628093
99 		 0.0

TOP 3 RANKED DOCUMENTS:
Document ID: 23
Content:
 white pea mixed finely chopped onion tomato chilli coriander coconut tamarind juice make delicacy traditionally eaten luchi deepfried flatbread radha ballabhi stuffed deepfried flatbread

Document ID: 92
Content:
 deepfried flatbread made wheat generous amount ghee clarified butter eaten curry especially fish curry

Document ID: 44
Content:
 vadas crisp fried doughnut made black gram lentil spice like cumin seed green chilli crispy deepfried snack usually served coconut chutney sambhar

********************************

********************************
QUERY NO

### Part C

In [29]:
# document representation
term_weights1 = get_term_weights(documents,tokens,'logNormal','invFreqProb')
term_weights2 = get_term_weights(documents,tokens,'logNormal','invFreqSmooth')
term_weights3 = get_term_weights(documents,tokens,'doubleNormalK','invFreqProb')
term_weights4 = get_term_weights(documents,tokens,'doubleNormalK','invFreqSmooth')

print('Word \t\tLN+Prob LN+Smooth DN+Prob DN+Smooth')
for i in range(len(term_weights1[0])):
  if term_weights1[0][i]!=0:
    print('{0: <12}'.format(list(tokens)[i]),'\t',round(term_weights1[0][i],3),'\t',round(term_weights2[0][i],3),'\t',round(term_weights3[0][i],3),'\t',round(term_weights4[0][i],3))

Word 		LN+Prob LN+Smooth DN+Prob DN+Smooth
architecture 	 3.17 	 3.459 	 1.506 	 1.643
lady         	 4.585 	 4.7 	 2.178 	 2.233
constructed  	 3.97 	 4.143 	 1.886 	 1.968
stunning     	 4.585 	 4.7 	 2.178 	 2.233
renowned     	 3.732 	 3.934 	 1.773 	 1.869
138          	 6.629 	 6.658 	 3.149 	 3.163
jharokhas    	 6.629 	 6.658 	 3.149 	 3.163
ostentatious 	 6.629 	 6.658 	 3.149 	 3.163
glass        	 3.97 	 4.143 	 1.886 	 1.968
property     	 6.629 	 6.658 	 3.149 	 3.163
chand        	 6.629 	 6.658 	 3.149 	 3.163
around       	 4.248 	 4.392 	 2.018 	 2.086
decorated    	 5.015 	 5.102 	 2.382 	 2.423
assortment   	 6.629 	 6.658 	 3.149 	 3.163
lake         	 4.585 	 4.7 	 2.178 	 2.233
museum       	 4.248 	 4.392 	 2.018 	 2.086
minister     	 5.015 	 5.102 	 2.382 	 2.423
mansion      	 6.629 	 6.658 	 3.149 	 3.163
including    	 3.338 	 3.598 	 1.585 	 1.709
till         	 4.585 	 4.7 	 2.178 	 2.233
archway      	 5.015 	 5.102 	 2.382 	 2.423
pichola      	 5.615 	 

### Part D - Compare rankings using different variants

In [38]:
#Assuming query terms occur in vocab
queries = [['deepfried','flatbread'],['standing','waterfront','pichola'], ['traditional','curry','pakoda','boiled','rice']]

for idx, query in enumerate(queries):
  rankings = generate_ranked_list(preprocessed_texts,tokens,term_weights1,query,'cosine') 
  print('********************************')
  print('QUERY NO:',idx,'\nQUERY:', query, '\n\nTOP 10 RANKINGS:',end='\n\n')
  print('Document ID\tEuclidean distance with query')
  for i in range(10):
    print(rankings[i][1],'\t\t',rankings[i][0])
  
  print('\nTOP 3 RANKED DOCUMENTS:')
  for i in range(3):
    print('Document ID:', rankings[i][1])
    print('Content:\n',' '.join(documents[rankings[i][1]]),end='\n\n')
  print('********************************\n')

********************************
QUERY NO: 0 
QUERY: ['deepfried', 'flatbread'] 

TOP 10 RANKINGS:

Document ID	Euclidean distance with query
23 		 0.5096252228022501
92 		 0.3516165262672819
44 		 0.1916852893740819
30 		 0.11193276351934545
91 		 0.09276117092475589
3 		 0.07603810841195924
85 		 0.04610845182167267
41 		 0.04437050023606348
13 		 0.03747416017247229
99 		 0.0

TOP 3 RANKED DOCUMENTS:
Document ID: 23
Content:
 white pea mixed finely chopped onion tomato chilli coriander coconut tamarind juice make delicacy traditionally eaten luchi deepfried flatbread radha ballabhi stuffed deepfried flatbread

Document ID: 92
Content:
 deepfried flatbread made wheat generous amount ghee clarified butter eaten curry especially fish curry

Document ID: 44
Content:
 vadas crisp fried doughnut made black gram lentil spice like cumin seed green chilli crispy deepfried snack usually served coconut chutney sambhar

********************************

********************************
QUERY NO

In [32]:
#Assuming query terms occur in vocab
queries = [['deepfried','flatbread'],['standing','waterfront','pichola'], ['traditional','curry','pakoda','boiled','rice']]

for idx, query in enumerate(queries):
  rankings = generate_ranked_list(preprocessed_texts,tokens,term_weights4,query,'euclidean') 
  print('********************************')
  print('QUERY NO:',idx,'\nQUERY:', query, '\n\nTOP 10 RANKINGS:',end='\n\n')
  print('Document ID\tEuclidean Distance with query')
  for i in range(10):
    print(rankings[i][1],'\t\t',rankings[i][0])
  
  print('\nTOP 3 RANKED DOCUMENTS:')
  for i in range(3):
    print('Document ID:', rankings[i][1])
    print('Content:\n',' '.join(documents[rankings[i][1]]),end='\n\n')
  print('********************************\n')

********************************
QUERY NO: 0 
QUERY: ['deepfried', 'flatbread'] 

TOP 10 RANKINGS:

Document ID	Euclidean Distance with query
92 		 95.83992353085887
96 		 96.06844676922334
3 		 96.07056332927066
21 		 96.08276694562484
23 		 96.1500357254336
33 		 96.20174365158046
75 		 96.25148723001715
73 		 96.27695276310855
87 		 96.30515777108285
9 		 96.32301083799965

TOP 3 RANKED DOCUMENTS:
Document ID: 92
Content:
 deepfried flatbread made wheat generous amount ghee clarified butter eaten curry especially fish curry

Document ID: 96
Content:
 delicious curry prepared gram flour dish gatte steamed gram flour dumpling cooked spicy curdbased gravy

Document ID: 3
Content:
 essentially fried roti flatbread delicious filling kathi roll roll wellloved snack kolkata paneer mutton chicken may used filling roll along egg

********************************

********************************
QUERY NO: 1 
QUERY: ['standing', 'waterfront', 'pichola'] 

TOP 10 RANKINGS:

Document ID	Euclide